# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import patsy

%matplotlib inline

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [4]:
df.drop(columns='Unnamed: 0', inplace=True)
df.head()

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [5]:
counts = {col: df[col].value_counts() for col in df.columns}
for col, val in counts.items():
    print(f'\n{col} : {val}')


data_ref : data_ref
2015-01-01    1000
2015-02-01    1000
2015-03-01    1000
2015-04-01    1000
2015-05-01    1000
2015-06-01    1000
2015-07-01    1000
2015-08-01    1000
2015-09-01    1000
2015-10-01    1000
2015-11-01    1000
2015-12-01    1000
2016-01-01    1000
2016-02-01    1000
2016-03-01    1000
Name: count, dtype: int64

id_cliente : id_cliente
9948     6
9351     6
8635     6
6356     6
5573     6
        ..
348      1
12243    1
677      1
292      1
14651    1
Name: count, Length: 9845, dtype: int64

sexo : sexo
F    10119
M     4881
Name: count, dtype: int64

posse_de_veiculo : posse_de_veiculo
False    9140
True     5860
Name: count, dtype: int64

posse_de_imovel : posse_de_imovel
True     10143
False     4857
Name: count, dtype: int64

qtd_filhos : qtd_filhos
0     10376
1      3037
2      1376
3       185
4        17
7         5
14        2
5         2
Name: count, dtype: int64

tipo_renda : tipo_renda
Assalariado         7633
Empresário          3508
Pensionista      

In [6]:
y, X = patsy.dmatrices('np.log(renda) ~ C(sexo, Treatment("F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment("Assalariado")) + C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado")) + C(tipo_residencia, Treatment("Casa")) + idade + tempo_emprego + qt_pessoas_residencia', data=df)

In [7]:
reg = sm.OLS(y, X).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sat, 29 Jun 2024   Prob (F-statistic):               0.00
Time:                        01:31:02   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5264      0.219     29.853      0.000       6.098       6.955
C(sexo, Treatment("F"))[T.M]                                    0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                                        0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]                                         0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment("Secundário"))[T.Primário]                0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil, Treatment("Casado"))[T.Separado]                0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment("Casa"))[T.Aluguel]                0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment("Casa"))[T.Com os pais]            0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment("Casa"))[T.Comunitário]           -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment("Casa"))[T.Estúdio]                0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment("Casa"))[T.Governamental]         -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos 

In [8]:
#retirado educacao
y, X = patsy.dmatrices('np.log(renda) ~ C(sexo, Treatment("F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment("Assalariado")) + C(estado_civil, Treatment("Casado")) + C(tipo_residencia, Treatment("Casa")) + idade + tempo_emprego + qt_pessoas_residencia', data=df)

reg = sm.OLS(y, X).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     340.0
Date:                Sat, 29 Jun 2024   Prob (F-statistic):               0.00
Time:                        01:31:02   Log-Likelihood:                -13601.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12406   BIC:                         2.740e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5899      0.219     30.108      0.000       6.161       7.019
C(sexo, Treatment("F"))[T.M]                                    0.7800      0.015     53.208      0.000       0.751       0.809
posse_de_veiculo[T.True]                                        0.0533      0.014      3.769      0.000       0.026       0.081
posse_de_imovel[T.True]                                         0.0856      0.014      6.110      0.000       0.058       0.113
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2988      0.242      1.237      0.216      -0.175       0.772
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1651      0.015     11.079      0.000       0.136       0.194
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.2513      0.242     -1.039      0.299      -0.725       0.223
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0754      0.022      3.400      0.001       0.032       0.119
C(estado_civil, Treatment("Casado"))[T.Separado]                0.3238      0.112      2.903      0.004       0.105       0.542
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.2616      0.109      2.396      0.017       0.048       0.476
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0374      0.025     -1.481      0.139      -0.087       0.012
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.3601      0.116      3.107      0.002       0.133       0.587
C(tipo_residencia, Treatment("Casa"))[T.Aluguel]                0.0369      0.054      0.680      0.496      -0.069       0.143
C(tipo_residencia, Treatment("Casa"))[T.Com os pais]            0.0185      0.029      0.627      0.530      -0.039       0.076
C(tipo_residencia, Treatment("Casa"))[T.Comunitário]           -0.0403      0.091     -0.443      0.658      -0.218       0.138
C(tipo_residencia, Treatment("Casa"))[T.Estúdio]                0.1225      0.084      1.456      0.145      -0.042       0.287
C(tipo_residencia, Treatment("Casa"))[T.Governamental]         -0.0209      0.039     -0.535      0.593      -0.098       0.056
qtd_filhos                                                     -0.2567      0.108     -2.372      0.018      -0.469      -0.045
idade                                                           0.0049      0.001      6.185      0.000       0.003       0.006
tempo_emprego                                                   0.0617      0.001     59.327      0.000       0.060       0.064
qt_pessoas_residencia                                           0.2846      0.108      2.634      0.008       0.073       0.496
===========

In [9]:
#retirado tipo de residencia
y, X = patsy.dmatrices('np.log(renda) ~ C(sexo, Treatment("F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment("Assalariado")) + C(estado_civil, Treatment("Casado")) + idade + tempo_emprego + qt_pessoas_residencia', data=df)

reg = sm.OLS(y, X).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     453.1
Date:                Sat, 29 Jun 2024   Prob (F-statistic):               0.00
Time:                        01:31:02   Log-Likelihood:                -13603.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12411   BIC:                         2.736e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5951      0.219     30.152      0.000       6.166       7.024
C(sexo, Treatment("F"))[T.M]                                    0.7819      0.015     53.480      0.000       0.753       0.811
posse_de_veiculo[T.True]                                        0.0535      0.014      3.789      0.000       0.026       0.081
posse_de_imovel[T.True]                                         0.0848      0.014      6.172      0.000       0.058       0.112
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2998      0.242      1.241      0.215      -0.174       0.773
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1655      0.015     11.120      0.000       0.136       0.195
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.2540      0.242     -1.051      0.293      -0.728       0.220
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0769      0.022      3.472      0.001       0.033       0.120
C(estado_civil, Treatment("Casado"))[T.Separado]                0.3241      0.112      2.907      0.004       0.106       0.543
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.2623      0.109      2.403      0.016       0.048       0.476
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0361      0.025     -1.431      0.153      -0.086       0.013
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.3588      0.116      3.097      0.002       0.132       0.586
qtd_filhos                                                     -0.2560      0.108     -2.366      0.018      -0.468      -0.044
idade                                                           0.0048      0.001      6.147      0.000       0.003       0.006
tempo_emprego                                                   0.0617      0.001     59.407      0.000       0.060       0.064
qt_pessoas_residencia                                           0.2839      0.108      2.629      0.009       0.072       0.496
==============================================================================
Omnibus:                        1.149   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.563   Jarque-Bera (JB):                1.121
Skew:                           0.021   Prob(JB):                        0.571
Kurtosis:                       3.019   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there a

In [10]:
#retirado tipo de renda
y, X = patsy.dmatrices('np.log(renda) ~ C(sexo, Treatment("F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(estado_civil, Treatment("Casado")) + idade + tempo_emprego + qt_pessoas_residencia', data=df)

reg = sm.OLS(y, X).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     600.2
Date:                Sat, 29 Jun 2024   Prob (F-statistic):               0.00
Time:                        01:31:02   Log-Likelihood:                -13666.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12415   BIC:                         2.745e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            6.7005      0.220     30.511      0.000       6.270       7.131
C(sexo, Treatment("F"))[T.M]                         0.7715      0.015     52.631      0.000       0.743       0.800
posse_de_veiculo[T.True]                             0.0565      0.014      3.985      0.000       0.029       0.084
posse_de_imovel[T.True]                              0.0868      0.014      6.288      0.000       0.060       0.114
C(estado_civil, Treatment("Casado"))[T.Separado]     0.3096      0.112      2.763      0.006       0.090       0.529
C(estado_civil, Treatment("Casado"))[T.Solteiro]     0.2533      0.110      2.309      0.021       0.038       0.468
C(estado_civil, Treatment("Casado"))[T.União]       -0.0281      0.025     -1.110      0.267      -0.078       0.022
C(estado_civil, Treatment("Casado"))[T.Viúvo]        0.3317      0.116      2.850      0.004       0.104       0.560
qtd_filhos                                          -0.2443      0.109     -2.247      0.025      -0.457      -0.031
idade                                                0.0046      0.001      5.783      0.000       0.003       0.006
tempo_emprego                                        0.0611      0.001     59.136      0.000       0.059       0.063
qt_pessoas_residencia                                0.2677      0.109      2.466      0.014       0.055       0.480
==============================================================================
Omnibus:                        1.331   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.514   Jarque-Bera (JB):                1.311
Skew:                           0.024   Prob(JB):                        0.519
Kurtosis:                       3.013   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
#retirado qtd de pessoas na residencia
y, X = patsy.dmatrices('np.log(renda) ~ C(sexo, Treatment("F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tempo_emprego', data=df)

reg = sm.OLS(y, X).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1098.
Date:                Sat, 29 Jun 2024   Prob (F-statistic):               0.00
Time:                        01:31:02   Log-Likelihood:                -13673.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12420   BIC:                         2.741e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        7.2223      0.034    213.646      0.000       7.156       7.289
C(sexo, Treatment("F"))[T.M]     0.7688      0.015     52.768      0.000       0.740       0.797
posse_de_veiculo[T.True]         0.0561      0.014      3.986      0.000       0.029       0.084
posse_de_imovel[T.True]          0.0866      0.014      6.278      0.000       0.060       0.114
qtd_filhos                       0.0239      0.009      2.767      0.006       0.007       0.041
idade                            0.0049      0.001      6.399      0.000       0.003       0.006
tempo_emprego                    0.0610      0.001     59.084      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.243   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.537   Jarque-Bera (JB):                1.216
Skew:                           0.022   Prob(JB):                        0.545
Kurtosis:                       3.020   Cond. No.                         223.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# Pela analise de R2, R2 ajustado e AIC, nao houve grandes melhoras no modelo.
# Mas o proprio summary aponta que com o alto numero de variaveis existe um problema de multicolinearidade
# entao precisamos identificar as variaveis irrelevantes com o p-value e eliminá-las do modelo.